### 1. Datenimport & Initiale Inspektion
Laden der Rohdaten (Zebra-GPS & NASA-Wetterdaten). Umgehen des NASA-Metadaten-Headers durch `skiprows`. Erste Prüfung der Datenstruktur, Dimensionen und Datentypen mittels `.info()` und `.head()`.

In [2]:
import numpy as np
import pandas as pd

In [3]:
dateipfad1 = './data/Burchells Zebra Migration Data.csv'
dateipfad2 = './data/POWER Daily Data 2007-2009.csv'

zebra= pd.read_csv(dateipfad1)
weather= pd.read_csv(dateipfad2, header=14)


/var/folders/8v/gy_42drs2hn6m4lfq1g21nr40000gn/T/ipykernel_60549/341517687.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  zebra= pd.read_csv(dateipfad1)


In [4]:
zebra.head()

,event-id,visible,timestamp,location-long,location-lat,manually-marked-outlier,sensor-type,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier,study-name,utm-easting,utm-northing,utm-zone,study-timezone,study-local-timestamp
0,210260446,True,2007-10-25 00:02:11.000,23.523616,-19.391531,NaN,gps,Equus burchellii,3864,Z3864,Migratory Burchell's zebra (Equus burchellii) ...,765050.173519,7.853911e+06,34S,Central African Time,2007-10-25 02:02:11.000
1,210260447,True,2007-10-25 01:01:49.000,23.523497,-19.391684,NaN,gps,Equus burchellii,3864,Z3864,Migratory Burchell's zebra (Equus burchellii) ...,765037.400397,7.853894e+06,34S,Central African Time,2007-10-25 03:01:49.000
2,210260448,True,2007-10-25 02:01:38.000,23.523463,-19.391651,NaN,gps,Equus burchellii,3864,Z3864,Migratory Burchell's zebra (Equus burchellii) ...,765033.860419,7.853898e+06,34S,Central African Time,2007-10-25 04:01:38.000
3,210260449,True,2007-10-25 03:02:18.000,23.523425,-19.391645,NaN,gps,Equus burchellii,3864,Z3864,Migratory Burchell's zebra (Equus burchellii) ...,765029.961121,7.853898e+06,34S,Central African Time,2007-10-25 05:02:18.000
4,210260450,True,2007-10-25 04:01:35.000,23.523434,-19.391541,NaN,gps,Equus burchellii,3864,Z3864,Migratory Burchell's zebra (Equus burchellii) ...,765031.042874,7.853910e+06,34S,Central African Time,2007-10-25 06:01:35.000


In [5]:
print(f"Geladen: {zebra.shape[0]} Zeilen, {zebra.shape[1]} Spalten")
print("\n")
zebra.info()

Geladen: 53794 Zeilen, 16 Spalten


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53794 entries, 0 to 53793
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   event-id                         53794 non-null  int64  
 1   visible                          53794 non-null  bool   
 2   timestamp                        53794 non-null  object 
 3   location-long                    53777 non-null  float64
 4   location-lat                     53777 non-null  float64
 5   manually-marked-outlier          1 non-null      object 
 6   sensor-type                      53794 non-null  object 
 7   individual-taxon-canonical-name  53794 non-null  object 
 8   tag-local-identifier             53794 non-null  int64  
 9   individual-local-identifier      53794 non-null  object 
 10  study-name                       53794 non-null  object 
 11  utm-easting                      53777 non-n

### 2. Identifikation & Entfernung von Ausreißern
Ein manuell markierter GPS-Fehler (`manually-marked-outlier = True`) mit unrealistischen Koordinaten wurde identifiziert. Die betroffene Zeile wurde entfernt, um die Datenintegrität zu wahren. Die redundante Markierungs-Spalte wurde anschließend gelöscht.

In [6]:
outlier_row = zebra[zebra['manually-marked-outlier'].notnull()]
print(outlier_row.T)

                                                                             12240
event-id                                                                 209298472
visible                                                                      False
timestamp                                                  2008-07-15 15:01:39.000
location-long                                                               -135.0
location-lat                                                             19.717682
manually-marked-outlier                                                       True
sensor-type                                                                    gps
individual-taxon-canonical-name                                   Equus burchellii
tag-local-identifier                                                          3743
individual-local-identifier                                                  Z3743
study-name                       Migratory Burchell's zebra (Equus burchellii) ...
utm-

In [7]:
zebra.describe()

,event-id,location-long,location-lat,tag-local-identifier,utm-easting,utm-northing
count,5.379400e+04,53777.000000,53777.000000,53794.000000,53777.000000,5.377700e+04
mean,2.094153e+08,24.457529,-20.112581,5119.377663,447519.050309,7.774203e+06
std,3.392140e+05,0.951067,0.543756,1294.828447,236244.477263,6.161720e+04
min,2.092416e+08,-135.000000,-20.811561,3743.000000,185502.003260,2.180240e+06
25%,2.092550e+08,23.620847,-20.569475,3864.000000,284702.923451,7.724225e+06
50%,2.092936e+08,24.873602,-20.407418,6399.000000,298554.434149,7.742152e+06
75%,2.093098e+08,25.020518,-19.437847,6405.000000,769809.289037,7.848646e+06
max,2.102632e+08,25.352691,19.717682,6407.000000,814471.248926,7.874175e+06


In [8]:
zebra_clean= zebra.copy()
zebra_clean= zebra_clean[zebra_clean['manually-marked-outlier'].isnull()]
zebra_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53793 entries, 0 to 53793
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   event-id                         53793 non-null  int64  
 1   visible                          53793 non-null  bool   
 2   timestamp                        53793 non-null  object 
 3   location-long                    53776 non-null  float64
 4   location-lat                     53776 non-null  float64
 5   manually-marked-outlier          0 non-null      object 
 6   sensor-type                      53793 non-null  object 
 7   individual-taxon-canonical-name  53793 non-null  object 
 8   tag-local-identifier             53793 non-null  int64  
 9   individual-local-identifier      53793 non-null  object 
 10  study-name                       53793 non-null  object 
 11  utm-easting                      53776 non-null  float64
 12  utm-northing           

In [9]:
zebra_clean = zebra_clean.drop('manually-marked-outlier', axis=1)

### 3. Datentyp-Korrektur (Zebra-Daten)
Konvertierung der Zeitstempel (`timestamp`, `study-local-timestamp`) vom Typ `object` (Text) in das `datetime64`-Format. Dies ist essenziell für die spätere Zeitreihenanalyse.

In [11]:
zebra_clean[['timestamp', 'study-local-timestamp']].info()

zebra_clean['timestamp'] = pd.to_datetime(zebra_clean['timestamp'])
zebra_clean['study-local-timestamp'] = pd.to_datetime(zebra_clean['study-local-timestamp'])

print("\n ------------------------------------------------------- \n")

zebra_clean[['timestamp', 'study-local-timestamp']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 53793 entries, 0 to 53793
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   timestamp              53793 non-null  datetime64[ns]
 1   study-local-timestamp  53793 non-null  datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 1.2 MB

 ------------------------------------------------------- 

<class 'pandas.core.frame.DataFrame'>
Index: 53793 entries, 0 to 53793
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   timestamp              53793 non-null  datetime64[ns]
 1   study-local-timestamp  53793 non-null  datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 1.2 MB


### 4. Behandlung fehlender Werte
Entfernung von 17 Zeilen mit fehlenden GPS-Koordinaten (`NaN` in `location-long/lat`). Da dies <0.04% des Datensatzes entspricht, wurde das Löschen (`dropna`) gegenüber der Interpolation bevorzugt, um die Genauigkeit der Messwerte zu garantieren.

In [14]:
zebra_clean.info()
zebra_clean[zebra_clean['location-long'].isnull()]

<class 'pandas.core.frame.DataFrame'>
Index: 53776 entries, 0 to 53793
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   event-id                         53776 non-null  int64         
 1   visible                          53776 non-null  bool          
 2   timestamp                        53776 non-null  datetime64[ns]
 3   location-long                    53776 non-null  float64       
 4   location-lat                     53776 non-null  float64       
 5   sensor-type                      53776 non-null  object        
 6   individual-taxon-canonical-name  53776 non-null  object        
 7   tag-local-identifier             53776 non-null  int64         
 8   individual-local-identifier      53776 non-null  object        
 9   study-name                       53776 non-null  object        
 10  utm-easting                      53776 non-null  float64       

,event-id,visible,timestamp,location-long,location-lat,sensor-type,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier,study-name,utm-easting,utm-northing,utm-zone,study-timezone,study-local-timestamp


In [13]:
zebra_clean = zebra_clean.dropna(subset=['location-long', 'location-lat'])
zebra_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53776 entries, 0 to 53793
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   event-id                         53776 non-null  int64         
 1   visible                          53776 non-null  bool          
 2   timestamp                        53776 non-null  datetime64[ns]
 3   location-long                    53776 non-null  float64       
 4   location-lat                     53776 non-null  float64       
 5   sensor-type                      53776 non-null  object        
 6   individual-taxon-canonical-name  53776 non-null  object        
 7   tag-local-identifier             53776 non-null  int64         
 8   individual-local-identifier      53776 non-null  object        
 9   study-name                       53776 non-null  object        
 10  utm-easting                      53776 non-null  float64       

### 5. Standardisierung & Selektion
Bereinigung der Spaltennamen (Ersetzen von Bindestrichen durch Unterstriche für bessere Handhabung). Entfernung irrelevanter Metadaten-Spalten (z.B. Sensor-Typ, Studien-Name), um den Datensatz auf die analytisch relevanten Features zu reduzieren.

In [153]:
zebra_clean.columns = [
    col.replace('-', '_') for col in zebra_clean.columns
]

zebra_clean = zebra_clean.drop([
    'sensor_type', 
    'individual_taxon_canonical_name', 
    'study_name', 
    'study_timezone',
    'study_local_timestamp', 
    'utm_easting', 
    'utm_northing', 
    'utm_zone'
], axis=1)


In [154]:
zebra_clean.info()
zebra_clean.head()

<class 'pandas.core.frame.DataFrame'>
Index: 53776 entries, 0 to 53793
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   event_id                     53776 non-null  int64         
 1   visible                      53776 non-null  bool          
 2   timestamp                    53776 non-null  datetime64[ns]
 3   location_long                53776 non-null  float64       
 4   location_lat                 53776 non-null  float64       
 5   tag_local_identifier         53776 non-null  int64         
 6   individual_local_identifier  53776 non-null  object        
dtypes: bool(1), datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 2.9+ MB


,event_id,visible,timestamp,location_long,location_lat,tag_local_identifier,individual_local_identifier
0,210260446,True,2007-10-25 00:02:11,23.523616,-19.391531,3864,Z3864
1,210260447,True,2007-10-25 01:01:49,23.523497,-19.391684,3864,Z3864
2,210260448,True,2007-10-25 02:01:38,23.523463,-19.391651,3864,Z3864
3,210260449,True,2007-10-25 03:02:18,23.523425,-19.391645,3864,Z3864
4,210260450,True,2007-10-25 04:01:35,23.523434,-19.391541,3864,Z3864


### 6. Vorbereitung der Wetterdaten (Feature Engineering)
Erstellung einer kompatiblen `date`-Spalte aus `YEAR` und `DOY` (Day of Year) im NASA-Datensatz. Konvertierung in `datetime64[ns]`, um als Schlüssel für den Merge zu dienen. Unnötige Spalten wurden entfernt.

In [155]:
print(f"Geladen: {weather.shape[0]} Zeilen, {weather.shape[1]} Spalten")
print("\n")
weather.info()
print("\n")
weather.head()

Geladen: 1062 Zeilen, 8 Spalten


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1062 entries, 0 to 1061
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   YEAR         1062 non-null   int64  
 1   DOY          1062 non-null   int64  
 2   T2M          1062 non-null   float64
 3   RH2M         1062 non-null   float64
 4   WS2M         1062 non-null   float64
 5   T2M_MAX      1062 non-null   float64
 6   T2M_MIN      1062 non-null   float64
 7   PRECTOTCORR  1062 non-null   float64
dtypes: float64(6), int64(2)
memory usage: 66.5 KB




,YEAR,DOY,T2M,RH2M,WS2M,T2M_MAX,T2M_MIN,PRECTOTCORR
0,2007,1,23.06,68.05,4.28,27.45,18.48,16.83
1,2007,2,21.73,69.38,4.80,26.37,16.98,4.28
2,2007,3,24.96,51.50,3.94,32.28,17.67,0.33
3,2007,4,25.56,43.07,3.59,34.17,16.45,0.00
4,2007,5,26.76,52.45,3.62,34.38,18.56,0.78


In [156]:
weather['date'] = pd.to_datetime(weather['YEAR'].astype(str) + '-' + weather['DOY'].astype(str), format='%Y-%j')
weather = weather.drop(['YEAR', 'DOY'], axis=1)

In [157]:
weather.info()
print("\n")
weather.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1062 entries, 0 to 1061
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   T2M          1062 non-null   float64       
 1   RH2M         1062 non-null   float64       
 2   WS2M         1062 non-null   float64       
 3   T2M_MAX      1062 non-null   float64       
 4   T2M_MIN      1062 non-null   float64       
 5   PRECTOTCORR  1062 non-null   float64       
 6   date         1062 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(6)
memory usage: 58.2 KB




,T2M,RH2M,WS2M,T2M_MAX,T2M_MIN,PRECTOTCORR,date
0,23.06,68.05,4.28,27.45,18.48,16.83,2007-01-01
1,21.73,69.38,4.80,26.37,16.98,4.28,2007-01-02
2,24.96,51.50,3.94,32.28,17.67,0.33,2007-01-03
3,25.56,43.07,3.59,34.17,16.45,0.00,2007-01-04
4,26.76,52.45,3.62,34.38,18.56,0.78,2007-01-05


### 7. Datenintegration (Merge)
Vorbereitung des Zebra-Datensatzes durch Extraktion des Datums und explizite Typumwandlung.
Zusammenführung der Datensätze mittels `Left Join`.
* **Ziel:** Anreicherung jedes Zebra-GPS-Punktes mit den täglichen Wetterdaten.
* **Ergebnis:** Ein konsistenter, `clean` Datensatz (`df_final`), bereit für die Analyse.

In [158]:
zebra_clean['date'] = zebra_clean['timestamp'].dt.date
zebra_clean['date'] = pd.to_datetime(zebra_clean['date'])

In [159]:
df_final = zebra_clean.merge(weather, on='date', how='left')

In [160]:
df_final = df_final.drop('date', axis=1)

print("--- Finaler Merged DataFrame ---")
print(df_final.info())

--- Finaler Merged DataFrame ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53776 entries, 0 to 53775
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   event_id                     53776 non-null  int64         
 1   visible                      53776 non-null  bool          
 2   timestamp                    53776 non-null  datetime64[ns]
 3   location_long                53776 non-null  float64       
 4   location_lat                 53776 non-null  float64       
 5   tag_local_identifier         53776 non-null  int64         
 6   individual_local_identifier  53776 non-null  object        
 7   T2M                          53776 non-null  float64       
 8   RH2M                         53776 non-null  float64       
 9   WS2M                         53776 non-null  float64       
 10  T2M_MAX                      53776 non-null  float64       
 11  T2M_MIN 

In [162]:
df_final.head()

,event_id,visible,timestamp,location_long,location_lat,tag_local_identifier,individual_local_identifier,T2M,RH2M,WS2M,T2M_MAX,T2M_MIN,PRECTOTCORR
0,210260446,True,2007-10-25 00:02:11,23.523616,-19.391531,3864,Z3864,29.47,36.09,5.03,37.2,21.94,0.5
1,210260447,True,2007-10-25 01:01:49,23.523497,-19.391684,3864,Z3864,29.47,36.09,5.03,37.2,21.94,0.5
2,210260448,True,2007-10-25 02:01:38,23.523463,-19.391651,3864,Z3864,29.47,36.09,5.03,37.2,21.94,0.5
3,210260449,True,2007-10-25 03:02:18,23.523425,-19.391645,3864,Z3864,29.47,36.09,5.03,37.2,21.94,0.5
4,210260450,True,2007-10-25 04:01:35,23.523434,-19.391541,3864,Z3864,29.47,36.09,5.03,37.2,21.94,0.5


In [163]:
output_path = './data/zebra_weather_cleaned.csv'

df_final.to_csv(output_path, index=False)
